In [6]:
import pandas as pd
import numpy as np
from numpy.random import default_rng

# Introduction
This notebook explores the classic percolation model for bushfire modelling and considers the Poisson distribution later.

In [119]:
def create_lattice(p, N, mode):
    rng = default_rng()
        
    if mode == 'bernoulli':
        x = rng.uniform(low=0, high=1, size=(N, N))
        y = x < p
    elif mode == 'poisson':
        y = rng.poisson(lam=p, size=(N, N))
    return y.astype(int)

# check if node contains fuel
def burnable(node, lattice):
    return lattice[node] > 0

# returns false if index is outside of lattice
def in_lattice(index, lattice):
    m, n = lattice.shape
    i, j = index
    if not 0 <= i <= m-1 or not 0 <= j <= n-1:
        return False
    else:
        return True

def get_neighbours(node, lattice):
    # get neighbours based on reach, i.e. node value
    # same for bernoulli and poisson modes, reach = L_inf distance
    nbs = []
    i, j = node
    reach = lattice[node]
#     k + l <= reach
    for k in range(reach + 1):
        for l in range(reach + 1 - k):
            for signk in [-1, 1]:
                for signl in [-1, 1]:
                    nb = (i + signk*k, j + signl* l)
                    if in_lattice(nb, lattice):
                        nbs.append(nb)
            
    # remove duplicates from k=0 or l=0
    nbs = list(set(nbs))
    # remove original node, case k=l=0
    if node in nbs:
        nbs.remove(node)
    return nbs
    
    

## Testing

In [120]:
x = create_lattice(.5, 5, 'bernoulli')
x

array([[1, 0, 0, 0, 1],
       [0, 0, 1, 1, 1],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 0, 1, 1, 0]])

In [121]:
y = create_lattice(.5, 5, 'poisson')
y

array([[0, 0, 1, 0, 1],
       [1, 0, 1, 1, 0],
       [0, 1, 1, 0, 2],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0]])

In [122]:
index = (0, 0)
print(in_lattice(index, x))
print(in_lattice((0, 4), x))


True
True


In [123]:
get_neighbours((0, 1), x)

[]

In [124]:
def simulate(p, N, mode):
    # initialize
    lattice = create_lattice(p, N, mode)
    matrices = [np.copy(lattice)]
    
    front = []
    archive = []
    
    # add burnable nodes in top row to front
    I, J = np.where(lattice[:1] > 0)
    for index in zip(I, J):
        front.append(index)
        
    # spread fire until front is emtpy
    while front:
        new_front = []
        
        for node in front:
            # get neighbours before burning
            nbs = get_neighbours(node, lattice)
            lattice[node] = -1
            
            # add burnable neighbours to new front
            for nb in nbs:
                if not nb in front and not nb in archive and burnable(nb, lattice):
                    new_front.append(nb)
        
        # update archive
        archive = archive + front
        # update front
        front = new_front
        
        # save snapshot of current lattice
        mat = np.copy(lattice)
        matrices.append(mat)
        
    return matrices

In [125]:
mats = simulate(.5, 5, 'bernoulli')

In [126]:
for mat in mats:
    print(mat, '\n')

[[1 0 1 0 1]
 [0 0 0 1 0]
 [0 1 1 1 1]
 [1 0 0 1 1]
 [1 1 1 0 1]] 

[[-1  0 -1  0 -1]
 [ 0  0  0  1  0]
 [ 0  1  1  1  1]
 [ 1  0  0  1  1]
 [ 1  1  1  0  1]] 

